In [72]:
import numpy as np
from tensorflow.keras.datasets import mnist,cifar10
from tqdm import tqdm
np.random.seed(42)
import warnings
warnings.filterwarnings("error")

weight_scaling = 0.1

In [73]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [74]:
def convert_categorical2one_hot(y: np.array) -> np.array:
    one_hot_matrix = np.zeros((y.size, y.max() + 1))
    one_hot_matrix[np.arange(y.size), y] = 1
    return one_hot_matrix

In [75]:
class GradientDescent:
    def __init__(self, learning_rate):
        self.learning_rate = learning_rate

    def update(self, layers):
        for layer in layers:
            weights_biases = layer.get_weights()
            gradients_weights_biases = layer.get_gradients()
            if weights_biases is None or gradients_weights_biases is None:
                continue

            (weights, biases) = weights_biases
            (derivative_weights, derivative_biases) = gradients_weights_biases
            layer.set_weights(
                weights=weights - self.learning_rate * derivative_weights,
                biases=biases - self.learning_rate * derivative_biases
            )

In [76]:
def generate_batches(x, y, batch_size):
    for i in range(0, x.shape[0], batch_size):
        yield (x.take(indices=range(
            i, min(i + batch_size, x.shape[0])), axis=0),
               y.take(indices=range(
                   i, min(i + batch_size, y.shape[0])), axis=0)
        )

In [77]:
def probability_to_one_hot(probabilities):
    class_index = np.argmax(probabilities, axis=1)
    one_hot_matrix = np.zeros_like(probabilities)
    one_hot_matrix[np.arange(probabilities.shape[0]), class_index] = 1
    return one_hot_matrix

In [78]:
def softmax_accuracy(y_hat, y):
    y_hat = probability_to_one_hot(y_hat)
    return (y_hat == y).all(axis=1).mean()

In [79]:
def softmax_cross_entropy(y_hat, y, epsilon=1e-20):
    n = y_hat.shape[0]
    return - np.sum(y * np.log(np.clip(y_hat, epsilon, 1.))) / n

In [80]:
class SequentialModel:
    def __init__(self, layers, ):
        self.layers = layers
        self.optimizer = GradientDescent(learning_rate=0.01)

        self.training_accuracies = []
        self.validation_accuracies = []
        self.train_losses = []
        self.validation_losses = []

    def initialize(self, x_train_sample):
        activation_sample = np.expand_dims(x_train_sample, axis=0)
        print("Configured Network-------")
        for i in range(len(self.layers)):
            print(f"Layer {i}: {self.layers[i].__class__.__name__}")
            self.layers[i].initialize(activation_sample)
            print(f"input dim: {activation_sample.shape}")
            activation_sample = self.layers[i].forward(a_previous=activation_sample)
            print(f"output dim: {activation_sample.shape}")

    def train(self, x_train, y_train, x_validation, y_validation, epoch_count, batch_size):
        self.initialize(x_train[0])
        for epoch in range(epoch_count):
            print(f"Epoch {epoch + 1}:")
            y_hat = np.zeros_like(y_train)
            for index, (x_batch, y_batch) in enumerate(tqdm(generate_batches(x=x_train, y=y_train, batch_size=batch_size),total=int(x_train.shape[0]/batch_size))):
                y_hat_batch = self.forward(x=x_batch)
                activation = y_hat_batch - y_batch
                self.backward(activation)
                self.update()
                number_start = index * batch_size
                number_end = number_start + y_hat_batch.shape[0]
                y_hat[number_start: number_end, :] = y_hat_batch

            print("calculating validation stats...")
            self.training_accuracies.append(softmax_accuracy(y_hat=y_hat, y=y_train))
            self.train_losses.append(softmax_cross_entropy(y_hat=y_hat, y=y_train))

            y_hat = self.forward(x_validation)
            validation_accuracy = softmax_accuracy(y_hat=y_hat, y=y_validation)
            self.validation_accuracies.append(validation_accuracy)
            validation_loss = softmax_cross_entropy(y_hat=y_hat, y=y_validation)
            self.validation_losses.append(validation_loss)

            print(f"Epoch {epoch + 1} validation loss {validation_loss} validation acc {validation_accuracy}")

    def predict(self, x):
        return self.forward(x)

    def forward(self, x):
        activation = x
        for layer in self.layers:
            # print(f"forward layer {layer.__class__.__name__}")
            # print(f"input dim: {activation.shape}")
            activation = layer.forward(a_previous=activation)
            # print(f"output dim: {activation.shape}")
        return activation

    def backward(self, x):
        activation = x
        for layer in reversed(self.layers):
            # print(f"backward layer {layer.__class__.__name__}")
            # print(f"input dim: {activation.shape}")
            activation = layer.backward(da_current=activation)
            # print(f"output dim: {activation.shape}")

    def update(self):
        self.optimizer.update(layers=self.layers)

In [81]:
class PoolingLayer:
    def __init__(self, pool_size, stride):
        self.pool_size = (pool_size,pool_size)
        self.stride = stride
        self.a = None
        self.cache = {}
    def initialize(self, activation):
        pass
    def get_weights(self):
        return None
    def get_gradients(self):
        return None
    def forward(self, a_previous):
        self.a = np.array(a_previous, copy=True)
        number_example_in_batch, height_input, width_input, channel_count = a_previous.shape
        height_pool, width_pool = self.pool_size
        height_output = 1 + (height_input - height_pool) // self.stride
        width_output = 1 + (width_input - width_pool) // self.stride
        output = np.zeros((number_example_in_batch, height_output, width_output, channel_count))
        for i in range(height_output):
            for j in range(width_output):
                height_start = i * self.stride
                height_end = height_start + height_pool
                width_start = j * self.stride
                width_end = width_start + width_pool
                a_previous_slice = a_previous[:, height_start:height_end, width_start:width_end, :]
                self.save_mask(x=a_previous_slice, coordinates=(i, j))
                output[:, i, j, :] = np.max(a_previous_slice, axis=(1, 2))
        return output

    def backward(self, da_current):
        output = np.zeros_like(self.a)
        _, height_output, width_output, _ = da_current.shape
        height_pool, width_pool = self.pool_size

        for i in range(height_output):
            for j in range(width_output):
                height_start = i * self.stride
                height_end = height_start + height_pool
                width_start = j * self.stride
                width_end = width_start + width_pool
                output[:, height_start:height_end, width_start:width_end, :] += da_current[:, i:i + 1, j:j + 1, :] * \
                                                                                self.cache[(i, j)]
        return output

    def save_mask(self, x, coordinates):
        mask = np.zeros_like(x)
        number_example_in_batch, height, width, channel_count = x.shape
        x = x.reshape(number_example_in_batch, height * width, channel_count)
        index = np.argmax(x, axis=1)
        number_index, channel_index = np.indices((number_example_in_batch, channel_count))
        mask.reshape(number_example_in_batch, height * width, channel_count)[number_index, index, channel_index] = 1
        self.cache[coordinates] = mask

In [82]:
class FlatteningLayer:
    def __init__(self):
        self.shape = ()
    def initialize(self, activation):
        pass
    def forward(self, a_previous):
        self.shape = a_previous.shape
        return np.ravel(a_previous).reshape(a_previous.shape[0], -1)

    def backward(self, da_current):
        return da_current.reshape(self.shape)

    def get_weights(self):
        return None
    def get_gradients(self):
        return None

In [83]:
class FullyConnectedLayer:
    def __init__(self, unit_count_current_layer):
        self.unit_count_current_layer = unit_count_current_layer
        #
        self.weights = None
        self.biases = None
        self.derivative_weights = None
        self.derivative_biases = None
        self.a_previous = None

    def initialize(self, activation):
        self.weights = np.random.randn(self.unit_count_current_layer, activation.shape[1]) * weight_scaling
        self.biases = np.random.randn(1, self.unit_count_current_layer) * weight_scaling

    def get_weights(self):
        return self.weights, self.biases

    def get_gradients(self):
        if self.derivative_weights is None or self.derivative_biases is None:
            return None
        return self.derivative_weights, self.derivative_biases

    def forward(self, a_previous):
        self.a_previous = np.array(a_previous, copy=True)
        return np.dot(a_previous, self.weights.T) + self.biases

    def backward(self, da_current):
        number_example_in_batch = self.a_previous.shape[0]
        self.derivative_weights = np.dot(da_current.T, self.a_previous) / number_example_in_batch
        self.derivative_biases = np.sum(da_current, axis=0, keepdims=True) / number_example_in_batch
        return np.dot(da_current, self.weights)

    def set_weights(self, weights, biases):
        self.weights = weights
        self.biases = biases

In [84]:
class ConvolutionLayer:
    def __init__(self, filters, kernel_shape, padding, stride):
        self.weights = None
        self.biases = None
        self.stride = stride
        self.padding = padding
        self.filters = filters
        self.kernel_shape = kernel_shape
        self.derivative_weights = None
        self.derivative_biases = None
        self.a_previous = None

    def initialize(self, activation):

        self.weights = np.random.randn(self.kernel_shape,self.kernel_shape,activation.shape[3], self.filters) * weight_scaling

        self.biases = np.random.randn(self.filters) * weight_scaling

    def get_weights(self):
        return self.weights, self.biases

    def set_weights(self, weights, biases):
        self.weights = weights
        self.biases = biases

    def get_gradients(self):
        if self.derivative_weights is None or self.derivative_biases is None:
            return None
        return self.derivative_weights, self.derivative_biases

    def forward(self, a_previous):
        self.a_previous = np.array(a_previous, copy=True)
        output_shape = self.calculate_output_dims(input_dims=a_previous.shape)
        number_examples_in_batch, height_input, width_of_input, _ = a_previous.shape
        _, height_output, width_output, _ = output_shape
        height_filter, width_filter, _, number_filter = self.weights.shape
        padding = np.full((4,), self.padding)
        a_previous_padding = self.pad(array=a_previous, padding=padding)
        output = np.zeros(output_shape)
        for i in range(height_output):
            for j in range(width_output):
                height_start = i * self.stride
                height_end = height_start + height_filter
                width_start = j * self.stride
                width_end = width_start + width_filter
                try:
                    output[:, i, j, :] = np.sum(a_previous_padding[:, height_start:height_end, width_start:width_end, :, np.newaxis] * self.weights[np.newaxis,:, :,:],axis=(1, 2, 3))
                except:
                    print("ERROR")
                    print(a_previous_padding)
                    raise KeyboardInterrupt
        return output + self.biases

    def backward(self, da_current):
        _, height_output, width_output, _ = da_current.shape
        number_example_in_batch, height_input, width_input, _ = self.a_previous.shape
        height_filter, width_filter, _, _ = self.weights.shape
        padding = np.full((4,), self.padding)
        a_previous_padding = self.pad(array=self.a_previous, padding=padding)
        output = np.zeros_like(a_previous_padding)
        self.derivative_biases = da_current.sum(axis=(0, 1, 2)) / number_example_in_batch
        self.derivative_weights = np.zeros_like(self.weights)
        for i in range(height_output):
            for j in range(width_output):
                height_start = i * self.stride
                height_end = height_start + height_filter
                width_start = j * self.stride
                width_end = width_start + width_filter
                output[:, height_start:height_end, width_start: width_end, :] += np.sum(
                    self.weights[np.newaxis, :, :, :, :] *
                    da_current[:, i:i + 1, j:j + 1, np.newaxis, :],
                    axis=4
                )
                self.derivative_weights += np.sum(
                    a_previous_padding[:, height_start: height_end, width_start: width_end, :, np.newaxis] *
                    da_current[:, i:i + 1, j:j + 1, np.newaxis, :],
                    axis=0
                )
        self.derivative_weights /= number_example_in_batch
        return output[:, padding[0]:padding[0] + height_input, padding[1]:padding[1] + width_input, :]

    @staticmethod
    def pad(array, padding):
        return np.pad(array=array, pad_width=((0, 0), (padding[0], padding[0]), (padding[1], padding[1]), (0, 0)),
                      mode='constant')

    def calculate_output_dims(self, input_dims):
        number_examples_in_batch, height_input, width_of_input, _ = input_dims
        height_filter, width_filter, _, number_filters = self.weights.shape

        height_output = (height_input - height_filter + 2 * self.padding) // self.stride + 1
        width_output = (width_of_input - width_filter + 2 * self.padding) // self.stride + 1

        return number_examples_in_batch, height_output, width_output, number_filters

In [85]:
class ActivationLayer:
    def __init__(self):
        self.z = None
    def initialize(self, activation):
        pass
    def forward(self, a_previous):
        self.z = np.maximum(0, a_previous)
        return self.z

    def backward(self, da_current):
        derivative_z = np.array(da_current, copy=True)
        derivative_z[self.z <= 0] = 0
        return derivative_z
    def get_weights(self):
        return None
    def get_gradients(self):
        return None

In [86]:
class SoftmaxLayer:
    def __init__(self):
        self.z = None
    def initialize(self, activation):
        pass
    def forward(self, a_previous):
        exponential = np.exp(a_previous - a_previous.max(axis=1, keepdims=True))
        self.z = exponential / np.sum(exponential, axis=1, keepdims=True)
        return self.z

    def backward(self, da_current):
        return da_current
    def get_weights(self):
        return None
    def get_gradients(self):
        return None

In [87]:
# number of samples in the train data set
N_TRAIN_SAMPLES = 5000
dataset = "mnist"

In [88]:
trainX, trainY, testX, testY = None, None, None, None
if dataset == "mnist":
    ((trainX, trainY), (testX, testY)) = mnist.load_data()
elif dataset == "cifar10":
    ((trainX, trainY), (testX, testY)) = cifar10.load_data()
    trainX = rgb2gray(trainX).astype(int)
    testX = rgb2gray(testX)

print("trainX shape:", trainX.shape)
print("trainY shape:", trainY.shape)
print("testX shape:", testX.shape)
print("testY shape:", testY.shape)

trainX shape: (60000, 28, 28)
trainY shape: (60000,)
testX shape: (10000, 28, 28)
testY shape: (10000,)


In [89]:
X_train = trainX[:N_TRAIN_SAMPLES, :, :]
Y_train = trainY[:N_TRAIN_SAMPLES]

test_size = int(testX.shape[0]/2)
X_test = testX[:test_size, :, :]
Y_test = testY[:test_size]

X_valid = testX[test_size:, :, :]
Y_valid = testY[test_size:]

In [90]:
X_train = X_train / 255
X_train = np.expand_dims(X_train, axis=3)
Y_train = convert_categorical2one_hot(Y_train)
X_test = X_test / 255
X_test = np.expand_dims(X_test, axis=3)
Y_test = convert_categorical2one_hot(Y_test)
X_valid = X_valid / 255
X_valid = np.expand_dims(X_valid, axis=3)
Y_valid = convert_categorical2one_hot(Y_valid)
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)
print("X_valid shape:", X_valid.shape)
print("Y_valid shape:", Y_valid.shape)

X_train shape: (5000, 28, 28, 1)
Y_train shape: (5000, 10)
X_test shape: (5000, 28, 28, 1)
Y_test shape: (5000, 10)
X_valid shape: (5000, 28, 28, 1)
Y_valid shape: (5000, 10)


In [91]:
all_layers = [
    ConvolutionLayer(filters=6, kernel_shape=5, stride=1, padding=2),
    ActivationLayer(),
    PoolingLayer(pool_size=2, stride=2),
    ConvolutionLayer(filters=12, kernel_shape=5, stride=1, padding=0),
    ActivationLayer(),
    PoolingLayer(pool_size=2, stride=2),
    ConvolutionLayer(filters=100, kernel_shape=5, stride=1, padding=0),
    ActivationLayer(),
    FlatteningLayer(),
    FullyConnectedLayer(unit_count_current_layer=10),
    SoftmaxLayer(),
]

In [92]:
model = SequentialModel(
    layers=all_layers
)

In [93]:
# model.initialize(X_train[0])

In [94]:
model.train(
    x_train=X_train,
    y_train=Y_train,
    x_validation=X_valid,
    y_validation=Y_valid,
    epoch_count=5,
    batch_size=32,
)

Configured Network-------
Layer 0: ConvolutionLayer
input dim: (1, 28, 28, 1)
output dim: (1, 28, 28, 6)
Layer 1: ActivationLayer
input dim: (1, 28, 28, 6)
output dim: (1, 28, 28, 6)
Layer 2: PoolingLayer
input dim: (1, 28, 28, 6)
output dim: (1, 14, 14, 6)
Layer 3: ConvolutionLayer
input dim: (1, 14, 14, 6)
output dim: (1, 10, 10, 12)
Layer 4: ActivationLayer
input dim: (1, 10, 10, 12)
output dim: (1, 10, 10, 12)
Layer 5: PoolingLayer
input dim: (1, 10, 10, 12)
output dim: (1, 5, 5, 12)
Layer 6: ConvolutionLayer
input dim: (1, 5, 5, 12)
output dim: (1, 1, 1, 100)
Layer 7: ActivationLayer
input dim: (1, 1, 1, 100)
output dim: (1, 1, 1, 100)
Layer 8: FlatteningLayer
input dim: (1, 1, 1, 100)
output dim: (1, 100)
Layer 9: FullyConnectedLayer
input dim: (1, 100)
output dim: (1, 10)
Layer 10: SoftmaxLayer
input dim: (1, 10)
output dim: (1, 10)
Epoch 1:


157it [00:28,  5.47it/s]                         


calculating validation stats...
Epoch 1 validation loss 2.1719849137897658 validation acc 0.319
Epoch 2:


 12%|█▏        | 18/156 [00:03<00:25,  5.35it/s]


KeyboardInterrupt: 

In [ ]:
# y_hat_test = model.predict(X_test)
# test_accuracy = softmax_accuracy(y_hat=y_hat_test, y=Y_test)
# print(f"Test set accuracy: {test_accuracy}")